In [1]:
import sys
import os
import json
import tensorflow as tf
import tqdm
from models.model import *
from ltv_utils import *
from losses.custom_loss import *
pd.set_option('display.float_format', '{:.4f}'.format)  # 保留10位小数，可调整
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)



def parse_function(serialized_example):
    feature_description = {
        'deviceid': tf.io.FixedLenFeature([], tf.string),
        'install_date': tf.io.FixedLenFeature([], tf.string),
        'dim_os_name1': tf.io.FixedLenFeature([], tf.string),
        'creative_classify1': tf.io.FixedLenFeature([], tf.string),
        'total_pay_amount1':  tf.io.FixedLenFeature([], tf.float32),
         'channel1': tf.io.FixedLenFeature([], tf.string),
        'b2_sale_amt_bias':  tf.io.FixedLenFeature([], tf.int64),
         'b2_sale_amt_7d': tf.io.FixedLenFeature([], tf.int64),
         'install_time': tf.io.FixedLenFeature([], tf.string),
        'install_order_diff':  tf.io.FixedLenFeature([], tf.int64),
        'all_install_order_7d_diff':  tf.io.FixedLenFeature([], tf.int64),
        'is_a1x_a33':  tf.io.FixedLenFeature([], tf.int64),
        'platform_label':  tf.io.FixedLenFeature([], tf.string),
        'user_dense_price_features': tf.io.FixedLenFeature([len(group_2_features['user_dense_price_features'])], tf.float32),
        'user_dense_duration_features': tf.io.FixedLenFeature([len(group_2_features['user_dense_duration_features'])], tf.float32),
        'user_dense_features': tf.io.FixedLenFeature([len(group_2_features['user_dense_features'])], tf.float32),
        'user_sparse_features': tf.io.FixedLenFeature([len(group_2_features['user_sparse_features'])], tf.float32)
    }
    example = tf.io.parse_single_example(serialized_example, feature_description)
    return example


# load tf records
group_2_features = read_feature_json_config('features/feature_list.json')

train_file_name = 'data/ltv_0522_0603_multi_window_model_train/part-r-00000'
valid_file_name = 'data/ltv_0522_0603_multi_window_model_valid/part-r-00000'

train_dataset, valid_dataset, _ = get_trian_valid_test_dateset(parse_function, 10000, train_file_name, valid_file_name)



user_dense_price_features = group_2_features['user_dense_price_features']
user_dense_duration_features = group_2_features['user_dense_duration_features']
user_dense_features = group_2_features['user_dense_features']
user_sparse_features = group_2_features['user_sparse_features']


In [2]:
MODEL_HOUR = 0 

In [3]:
def create_tf_dataset(dataset):
    sample_batch = next(iter(dataset))
    sample_data = {k: v for k, v in sample_batch.items() if k not in ['b2_sale_amt_7d', 'total_pay_amount1']}

    def generator():
        for batch in dataset:
            hour = tf.cast(tf.gather(batch['user_sparse_features'],  indices=0, axis = 1) - 1, tf.int64)    # shape: (batch_size,)
            b2_7d = tf.cast(tf.reshape(batch.pop('b2_sale_amt_7d'), (-1, 1)), tf.float32)
            b2_7d = tf.maximum(b2_7d, 0.0)
            
            total_amt_1h = tf.reshape(batch.pop('total_pay_amount1'), (-1, 1))

            # 只保留 hour 为 MODEL_HOUR 的记录
            hour_mask = tf.equal(hour, MODEL_HOUR)  # shape: (batch_size,)
            hour_mask = tf.reshape(hour_mask, (-1, 1))  # 广播成 (batch_size, 1)
            
            #  使用 hour_mask 筛选 batch 中的 对应小时窗口 
            selected_indices = tf.where(hour_mask)[:, 0]  # 获取 hour == 1 的样本索引
            batch = {k: tf.gather(v, selected_indices, axis=0) for k, v in batch.items()}  # 筛选 batch 中的样本
            b2_7d = tf.gather(b2_7d, selected_indices, axis=0)  # 保留 hour == 1 对应的标签

            y_true_packed = b2_7d
            
            yield batch, y_true_packed
        

    # 正确写法：output_signature 中保留每个字段的真实 shape
    output_signature = (
        {
            name: tf.TensorSpec(shape=(None,) + v.shape[1:], dtype=v.dtype)
            for name, v in sample_data.items()
        },
        tf.TensorSpec(shape=(None, 1), dtype=tf.float32)
    )

    return tf.data.Dataset.from_generator(generator, output_signature=output_signature)

In [18]:

from tensorflow.keras import Input, Model

# 构建输入层（只包含模型中用到的字段）
input_keys = ['user_dense_features', 'user_dense_price_features', 'user_dense_duration_features', 'user_sparse_features']
inputs = {
    key: Input(shape=(len(group_2_features[key]),), name=key)
    for key in input_keys
}


# 调用你自定义的特征拼接层
x = NO_Process_Layer(
    'user_dense_features',
    'user_dense_price_features',
    'user_dense_duration_features',
    'user_sparse_features'
)(inputs)
# 后续网络层
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
# x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)
# x = tf.keras.layers.BatchNormalization()(x)
output = tf.keras.layers.Dense(1)(x)

# 构建模型
model = Model(inputs=inputs, outputs=output)





# 自己实现的双口loss
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_rmse',  # 监控验证集上的 loss
    patience=3,          # 如果连续 3 轮没有改善，就停止训练
    restore_best_weights=True  # 训练结束后恢复到最优模型
)
loss_fn = tf.keras.losses.MeanSquaredError()
# loss_fn = UnifiedLTVLoss('mse')
# model.compile(loss=loss_fn, 
#               optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),  
#               metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')]
#              )

model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[
        tf.keras.metrics.RootMeanSquaredError(name='rmse')
    ]
)



In [19]:
model.fit(
    create_tf_dataset(train_dataset),
    epochs=15,
    validation_data = create_tf_dataset(valid_dataset),
    callbacks= [early_stopping]
) 

Epoch 1/15
248/248 [==============================] - 23s 90ms/step - loss: 58185.0430 - rmse: 241.2158 - val_loss: 61679.0352 - val_rmse: 248.3526
Epoch 2/15
248/248 [==============================] - 22s 88ms/step - loss: 55592.3008 - rmse: 235.7802 - val_loss: 60845.9102 - val_rmse: 246.6696
Epoch 3/15
248/248 [==============================] - 22s 88ms/step - loss: 55279.1602 - rmse: 235.1152 - val_loss: 60378.6641 - val_rmse: 245.7207
Epoch 4/15
248/248 [==============================] - 22s 89ms/step - loss: 55005.8477 - rmse: 234.5332 - val_loss: 60011.7383 - val_rmse: 244.9729
Epoch 5/15
248/248 [==============================] - 22s 88ms/step - loss: 54790.9570 - rmse: 234.0747 - val_loss: 59804.2969 - val_rmse: 244.5492
Epoch 6/15
248/248 [==============================] - 22s 90ms/step - loss: 54569.6523 - rmse: 233.6015 - val_loss: 59584.5898 - val_rmse: 244.0995
Epoch 7/15
248/248 [==============================] - 22s 89ms/step - loss: 54374.1211 - rmse: 233.1826 - val_lo

In [21]:
res = model.predict(create_tf_dataset(valid_dataset))
res.sum()

36/36 [==============================] - 3s 75ms/step


4645680.0

In [ ]:
res_train = model.predict(create_tf_dataset(train_dataset))

    240/Unknown - 18s 74ms/step

In [ ]:
res_train.sum()

In [12]:
shape_batch = 0 
label_sum = 0
for batch, y in create_tf_dataset(train_dataset):
    shape_batch += batch['b2_sale_amt_bias'].shape[0]
    print(batch['b2_sale_amt_bias'].shape[0])
    min_value = tf.reduce_sum(y )
    label_sum += min_value
print(shape_batch, label_sum)

2071
2011
1937
2018
1980
2024
2036
1981
2034
2007
1953
1968
2095
1922
2003
1969
2046
2065
1977
2012
2062
2121
1967
1905
2059
1980
1947
1997
2077
1985
1962
2037
2044
1953
1981
2006
2057
1979
1992
2013
1976
2044
1980
1950
2049
2132
1963
2119
2004
1968
2010
1891
2004
2098
1926
1981
1959
2002
2059
1922
1951
2124
1855
2008
2028
2058
1963
2000
2022
1923
2061
2034
2035
1994
2038
2040
2007
2065
1991
1933
2050
1993
1938
2098
2022
2022
1908
2016
1971
2066
1960
1985
2072
1979
1924
2094
1995
1941
2084
2010
2017
2045
2005
1989
1939
2021
2033
2023
2031
2004
2000
2059
1924
1973
1958
2049
2057
2009
2083
1922
1959
1933
1917
2060
2055
1966
1991
2014
1964
2014
2012
1951
1999
1982
2092
1920
2006
2043
2016
2064
2008
1993
2041
1965
2030
2071
1984
1866
1994
1968
1992
2053
1912
1976
2032
1984
1996
2035
2039
1956
1915
1913
1956
2061
1976
2072
2003
2024
1960
2037
1866
1976
2048
1946
1991
2059
1972
1995
2021
2062
2016
1998
1952
2038
2019
1967
1979
2012
1993
2016
2003
1930
2005
1961
1947
1943
1975
2027
2005
1871
